# Download Sentinel-2 data
* Iterate over each region in the supplied `regions.geojson` input 
* For each region, query Planetary Computer STAC database and load scenes into a data cube
  * Group scenes by solar date and subset to area provided in `regions.geojson`
* Save each scene by solar date by region

Regions (**bolded** regions selected in this analysis)
* **Gambia-flooding-8-11-2022**
* Hurricane-Fiona-9-19-2022
* Hurricane-Ian-9-26-2022
* **Indonesia-Earthquake22**
* **Kahramanmaras-turkey-earthquake-23**
* New-Zealand-Flooding22
* New-Zealand-Flooding23
* Sudan-flooding-8-22-2022
* **afghanistan-earthquake22**
* **cyclone-emnati22**
* kentucky-flooding-7-29-2022
* pakistan-flooding22
* southafrica-flooding22
* tonga-volcano21
* **volcano-indonesia21**
* yellowstone-flooding22
* **baltimore-nd**

In [18]:
# Standard library imports
import json
import os
from pathlib import Path

# Third-party imports
import dask
import dask.distributed
import dask.utils
from datacube.utils.cog import write_cog
from dotenv import load_dotenv
import geopandas as gpd
import numpy as np
from odc.stac import configure_rio, stac_load
import pandas as pd
import planetary_computer as pc
from pystac_client import Client
import rasterio as rio
from rasterio.mask import mask as rio_mask
from rasterio.session import AWSSession
import xarray as xr
from IPython.display import display

# Local imports
from utils import to_float


print("Load environment variables from .env file.")
load_dotenv()
USGS_API_KEY = os.environ["USGS_API_KEY"]
USGS_TOKEN_NAME = os.environ["USGS_TOKEN_NAME"]
USGS_USERNAME = os.environ["USGS_USERNAME"]
USGS_PASSWORD = os.environ["USGS_PASSWORD"]
AWS_ACCESS_KEY = os.environ["AWS_ACCESS_KEY"]
AWS_SECRET_KEY = os.environ["AWS_SECRET_KEY"]
NASA_EARTHDATA_S3_ACCESS_KEY = os.environ["NASA_EARTHDATA_S3_ACCESS_KEY"]
NASA_EARTHDATA_S3_SECRET_KEY = os.environ["NASA_EARTHDATA_S3_SECRET_KEY"]
NASA_EARTHDATA_S3_SESSION = os.environ["NASA_EARTHDATA_S3_SESSION"]
NASA_EARTHDATA_USERNAME = os.environ["NASA_EARTHDATA_USERNAME"]
NASA_EARTHDATA_PASSWORD = os.environ["NASA_EARTHDATA_PASSWORD"]

DATA_DIR = Path(r"C:\Users\Peter\gh\rasmussen-705.603\data\FinalProject")
RES = 10
STAC_ENDPOINT = "https://planetarycomputer.microsoft.com/api/stac/v1"
COLLECTIONS = ["sentinel-2-l2a"]
COLLECTION_BANDS = ["blue", "green", "red", "nir08", "swir16", "swir22", "qa"]
OUTPUT_BANDS = ["blue", "green", "red", "nir08", "swir16", "swir22", "ndvi", "qa"]

os.environ["GDAL_DISABLE_READDIR_ON_OPEN"] = "FALSE"

Load environment variables from .env file.


In [19]:
# Define directory paths
raw_dir = DATA_DIR / "raw"
interim_dir = DATA_DIR / "interim"
processed_dir = DATA_DIR / "processed"
region_dir = raw_dir / "regions"
dst_dir = interim_dir / "cogs"
dst_dir.mkdir(exist_ok=True, parents=True)

In [20]:
print("Load input region geojson and config.")
with open(raw_dir / "cfg.json")  as f:
    cfg = json.load(f)
regions = gpd.read_file(raw_dir / "regions.geojson")
regions["time_range"] = regions["s2_start"] + "/" + regions["s2_end"]

Load input region geojson and config.


In [21]:
print("Instantiate dask client.")
client = dask.distributed.Client()
configure_rio(cloud_defaults=True, client=client)
display(client)

Instantiate dask client.


c:\Users\Peter\anaconda3\envs\torchgeo\lib\site-packages\distributed\node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 58646 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:58646/status,
Dashboard: http://127.0.0.1:58646/status,Workers: 4
Total threads: 16,Total memory: 15.93 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:58647,Workers: 4
Dashboard: http://127.0.0.1:58646/status,Total threads: 16
Started: Just now,Total memory: 15.93 GiB
Comm: tcp://127.0.0.1:58667,Total threads: 4
Dashboard: http://127.0.0.1:58669/status,Memory: 3.98 GiB
Nanny: tcp://127.0.0.1:58650,


In [22]:
# event_key = regions.loc[index]["event_key"]
# time_range = regions.loc[index]["time_range"]
# geom_4326 = json.loads(regions.to_crs(4326).to_json())["features"][index]["geometry"]
# maxar_event = regions.loc[index]["maxar_event"]
# bbox_4326 = regions.bounds.loc[index].values.tolist()
# print(f"[{event_key}]: {time_range}.")

# print(f"[{event_key}]: Search catalog.")
# catalog = Client.open(STAC_ENDPOINT)
# query = catalog.search(
#     collections=COLLECTIONS,
#     datetime=time_range,
#     bbox=bbox_4326,
# )

# items = list(query.get_items())
# print(f"[{event_key}]: Found {len(items):d} items.")

# print(f"[{event_key}]: Load items into data cube.")
# xx = stac_load(
#     items,
#     bands=COLLECTION_BANDS,
#     resolution=RES,
#     chunks={"x": 1028, "y": 1028},
#     stac_cfg=cfg,
#     patch_url=pc.sign,
#     crs="utm",
#     bbox=bbox_4326,
#     fail_on_error=False,   
# )
# nir08 = to_float(xx.nir08)
# red = to_float(xx.red)
# ndvi = ((nir08 - red) / (nir08 + red)).fillna(0) * 10000
# xx["ndvi"] = ndvi

In [23]:
for index in regions.index.values:
    
    event_key = regions.loc[index]["event_key"]
    time_range = regions.loc[index]["time_range"]
    geom_4326 = json.loads(regions.to_crs(4326).to_json())["features"][index]["geometry"]
    maxar_event = regions.loc[index]["maxar_event"]
    bbox_4326 = regions.bounds.loc[index].values.tolist()
    print(f"[{event_key}]: {time_range}.")
    
    print(f"[{event_key}]: Search catalog.")
    catalog = Client.open(STAC_ENDPOINT)
    query = catalog.search(
        collections=COLLECTIONS,
        datetime=time_range,
        bbox=bbox_4326,
    )
    items = list(query.get_items())
    print(f"[{event_key}]: Found {len(items)} items.")
    
    items = [item for item in items if item.properties["eo:cloud_cover"] < 30]
    print(f"[{event_key}]: Selected {len(items)} items.")
    
    print(f"[{event_key}]: Load items into data cube.")
    xx = stac_load(
        items,
        bands=COLLECTION_BANDS,
        resolution=RES,
        chunks={"x": 1028, "y": 1028},
        stac_cfg=cfg,
        patch_url=pc.sign,
        crs="utm",
        bbox=bbox_4326,
        fail_on_error=False,   
    )
    nir08 = to_float(xx.nir08)
    red = to_float(xx.red)
    ndvi = ((nir08 - red) / (nir08 + red)).fillna(0) * 10000
    xx["ndvi"] = ndvi
    # xx["qa"] = (xx["qa"] > 0).astype("uint8")
    
    print(f"[{event_key}]: Re-order bands.")
    xx = xx[OUTPUT_BANDS].astype(np.int32)
    n_files = len(xx.time.data)
    print(f"[{event_key}]: Write {n_files} TIFs.")

    for i in range(n_files):
        date = xx.isel(time=i).time.dt.strftime("%Y-%m-%d").data
        dst = dst_dir / f"{event_key}_{date}.tif"
        try:
            arr = xx.isel(time=i).to_array()
            write_cog(geo_im=arr, fname=dst, overwrite=False).compute()
            print(f"[{event_key}]: Wrote {dst.name}.")
        except Exception as e:
            print(f"[{event_key}]: Failed to write {dst.name}.")
            print(f"[{event_key}]: {e}")
    


[af-kharkamar-2022]: 2016-01-01/2023-04-29.
[af-kharkamar-2022]: Search catalog.
[af-kharkamar-2022]: Found 431 items.
[af-kharkamar-2022]: Selected 293 items.
[af-kharkamar-2022]: Load items into data cube.
[af-kharkamar-2022]: Re-order bands.
[af-kharkamar-2022]: Write 289 TIFs.
[af-kharkamar-2022]: Wrote af-kharkamar-2022_2016-01-15.tif.
[af-kharkamar-2022]: Wrote af-kharkamar-2022_2016-02-14.tif.
[af-kharkamar-2022]: Wrote af-kharkamar-2022_2016-03-15.tif.
[af-kharkamar-2022]: Wrote af-kharkamar-2022_2016-04-14.tif.
[af-kharkamar-2022]: Wrote af-kharkamar-2022_2016-04-27.tif.
[af-kharkamar-2022]: Wrote af-kharkamar-2022_2016-05-07.tif.
[af-kharkamar-2022]: Wrote af-kharkamar-2022_2016-06-16.tif.
[af-kharkamar-2022]: Wrote af-kharkamar-2022_2016-06-23.tif.
[af-kharkamar-2022]: Wrote af-kharkamar-2022_2016-07-13.tif.
[af-kharkamar-2022]: Wrote af-kharkamar-2022_2016-08-22.tif.
[af-kharkamar-2022]: Wrote af-kharkamar-2022_2016-09-11.tif.
[af-kharkamar-2022]: Wrote af-kharkamar-2022_20

In [16]:
len(), len(items)

(68, 164)

In [12]:
items[0]

id: S2A_MSIL2A_20230425T154811_R054_T18SUJ_20230426T005642
"bbox: [-77.33362, 38.73820959, -76.03812, 39.74549355]"
datetime: 2023-04-25T15:48:11.024000Z
platform: Sentinel-2A
proj:epsg: 32618
instruments: ['msi']
s2:mgrs_tile: 18SUJ
constellation: Sentinel 2
s2:granule_id: S2A_OPER_MSI_L2A_TL_MSFT_20230426T005643_A040949_T18SUJ_N05.09
eo:cloud_cover: 34.768704
s2:datatake_id: GS2A_20230425T154811_040949_N05.09


In [10]:
for item in items:
    print(item.id, item.properties["eo:cloud_cover"])

S2A_MSIL2A_20230425T154811_R054_T18SUJ_20230426T005642 34.768704
S2B_MSIL2A_20230420T154819_R054_T18SUJ_20230420T235434 4.933086
S2A_MSIL2A_20230415T154931_R054_T18SUJ_20230416T011210 74.1193
S2B_MSIL2A_20230410T154819_R054_T18SUJ_20230410T235132 0.620818
S2A_MSIL2A_20230405T154941_R054_T18SUJ_20230406T000529 96.978551
S2B_MSIL2A_20230331T154819_R054_T18SUJ_20230331T225628 99.998027
S2A_MSIL2A_20230326T154901_R054_T18SUJ_20230327T002946 6.762125
S2A_MSIL2A_20230316T155001_R054_T18SUJ_20230317T001957 7.517573
S2B_MSIL2A_20230311T154959_R054_T18SUJ_20230311T225202 79.351813
S2A_MSIL2A_20230306T155121_R054_T18SUJ_20230307T022139 51.21972
S2B_MSIL2A_20230301T155159_R054_T18SUJ_20230302T034609 27.722794
S2A_MSIL2A_20230224T155231_R054_T18SUJ_20230226T164939 0.071848
S2B_MSIL2A_20230219T155309_R054_T18SUJ_20230222T123839 58.45502
S2B_MSIL2A_20230219T155309_R054_T18SUJ_20230221T073854 58.45502
S2A_MSIL2A_20230214T155331_R054_T18SUJ_20230215T041247 49.67545
S2A_MSIL2A_20230204T155441_R054_T18S